In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [4]:
# read dataset 
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train_orig = pd.read_csv("/kaggle/input/titanic/train.csv")

In [5]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [6]:
# find correlation 
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [7]:
train=train.drop("Cabin", axis=1)
train=train.drop("Name", axis=1)

In [8]:
# test data 
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test_orig = pd.read_csv("/kaggle/input/titanic/test.csv")
test.drop("Cabin",axis=1,inplace=True)
test.drop("Name",axis=1,inplace=True)
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,male,34.5,0,0,330911,7.8292,Q
1,893,3,female,47.0,1,0,363272,7.0000,S
2,894,2,male,62.0,0,0,240276,9.6875,Q
3,895,3,male,27.0,0,0,315154,8.6625,S
4,896,3,female,22.0,1,1,3101298,12.2875,S


In [9]:
columns_to_onehot = ["Sex", "Embarked"]
columns_to_label=["Ticket"]
train=pd.get_dummies(train, columns=columns_to_onehot)
test=pd.get_dummies(test, columns=columns_to_onehot)

In [10]:
# get train and test data
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()
train["Ticket"]=le.fit_transform(train["Ticket"])
test["Ticket"]=le.fit_transform(test["Ticket"])
pi = test["PassengerId"]

train = train.drop("PassengerId", axis=1)
test = test.drop("PassengerId", axis=1)
X_train = train.drop("Survived", axis=1)
y_train = train["Survived"]
X_test = test
X_train

,Pclass,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,523,7.2500,0,1,0,0,1
1,1,38.0,1,0,596,71.2833,1,0,1,0,0
2,3,26.0,0,0,669,7.9250,1,0,0,0,1
3,1,35.0,1,0,49,53.1000,1,0,0,0,1
4,3,35.0,0,0,472,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,101,13.0000,0,1,0,0,1
887,1,19.0,0,0,14,30.0000,1,0,0,0,1
888,3,NaN,1,2,675,23.4500,1,0,0,0,1
889,1,26.0,0,0,8,30.0000,0,1,1,0,0


In [11]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as L

In [15]:
#build model 
model = Sequential(name='titanic_model')

model.add(L.InputLayer(input_shape=(11,))) # necessary to use model.summary()

model.add(L.Dense(2048, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(1024, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(512, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(256, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(64, activation='relu'))

model.add(L.Dense(32, activation='relu'))
model.add(L.Dense(1, activation='sigmoid')) # output layer, use sigmoid for binary

model.summary()

Model: "titanic_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2048)              24576     
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)             

In [ ]:
model = Sequential(name='titanic_model')

model.add(L.InputLayer(input_shape=(11,))) # necessary to use model.summary()

model.add(L.Dense(2048, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(1024, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(512, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(256, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(64, activation='relu'))

model.add(L.Dense(32, activation='relu'))
model.add(L.Dense(1, activation='sigmoid')) # output layer, use sigmoid for binary

model.summary()

In [18]:
preds = model.predict(test)
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
submission['Survived'] = [0 if pred < 0.5 else 1 for pred in preds]
submission.head(20)

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
5,897,1
6,898,1
7,899,1
8,900,1
9,901,1


In [19]:
from IPython.display import FileLink


submission.to_csv('submission.csv',index=False)
FileLink(r'submission.csv')

/kaggle/working/submission.csv